## 参考資料
- [詳細確率ロボティクスのgit](https://github.com/ryuichiueda/LNPR_BOOK_CODES)


In [1]:
# conect to drive on colab
# from google.colab import drive
# drive.mount("/content/drive")
# dir_path="./drive/MyDrive/Colab Notebooks/ProbabilisticRobotics/"
dir_path="./"



In [3]:
import sys
sys.path.append(dir_path)
from ideal_robot import *
from scipy.stats import expon,norm
import numpy as np
pi=np.pi

### 不確かさの実装
- 雑音(ノイズ)
  - 指数分布(expon)を確率でドロー、ドローしたらリセットして雑音を加える
- バイアス
  - ガウス分布(norm)から初期化時にドロー、ガウス分布の何%かをバイアスとして設定
- 

In [30]:
class Robot(IdealRobot):
    def __init__(self,pose,agent=None,sensor=None,color="black",noise_per_meter=5,noise_std=pi/60,bias_rate_stds=(0.1,0.1)):
        super().__init__(pose,agent,sensor,color)
        # noise
        self.noise_pdf=expon(scale=1.0/(1e-100+noise_per_meter))
        self.distance_until_noise=self.noise_pdf.rvs()
        self.theta_noise=norm(scale=noise_std)
        # bias
        self.bias_rate_nu=norm.rvs(loc=1.0,scale=bias_rate_stds[0])
        self.bias_rate_omega=norm.rvs(loc=1.0,scale=bias_rate_stds[1])
    
    def bias(self,nu,omega):
        return nu*self.bias_rate_nu,omega*self.bias_rate_omega
        
    def noise(self,pose,nu,omega,time_interval):
        self.distance_until_noise-=abs(nu)*time_interval+self.r*abs(omega)*time_interval
        if self.distance_until_noise <= 0.0:
            self.distance_until_noise += self.noise_pdf.rvs()
            pose[2] += self.theta_noise.rvs()
        return pose

    def one_step(self,time_interval):
        if not self.agent: return
        obs = self.sensor.data(self.pose) if self.sensor else None
        nu,omega = self.agent.decision(obs)
        nu,omega=self.bias(nu,omega)
        self.pose = self.state_transition(nu,omega,time_interval,self.pose)
        self.pose = self.noise(self.pose,nu,omega,time_interval)
    


In [32]:
world = World(10, 0.1)

circling = Agent(0.2, 10.0/180*pi)
no_bias_robot= IdealRobot( np.array([0, 0, 0]).T, sensor=None, agent=circling, color="gray" )
world.append(no_bias_robot)

biased_robot= Robot( np.array([0, 0, 0]).T, sensor=None, agent=circling, color="red" ,noise_per_meter=0,bias_rate_stds=(0.2,0.2))
world.append(biased_robot)

world.draw()